# Matching amico with cosmoDC2_small

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import GCRCatalogs
import numpy as np
from astropy.table import Table, vstack
from astropy.io import fits
from astropy.io import ascii
import pandas as pd

###clevar
import clevar
from clevar import ClCatalog
from clevar.match import get_matched_pairs
from clevar.match import output_matched_catalog

### Listing variables names if needed

In [ ]:
#print(cat.list_all_quantities())
#print(amico_data.colnames)

In [ ]:
cat_names = ['9943_map_associations', '9942_map_associations', '9816_map_associations', '9815_map_associations', '9814_map_associations', '9687_map_associations', '9686_map_associations',
             '9559_map_associations', '10450_map_associations', '10327_map_associations', '10326_map_associations', '10200_map_associations', '10199_map_associations',
             '10198_map_associations', '10072_map_associations', '10071_map_associations']

In [ ]:
tile_names = []
for i in range(len(cat_names)):
    tile_names.append(cat_names[i].split("_")[0])

### Loading catalogs and converting them to clevar tables

In [ ]:
cat_names = ['9943_map_associations', '9942_map_associations', '9816_map_associations', '9815_map_associations', '9814_map_associations', '9687_map_associations', '9686_map_associations',
             '9559_map_associations', '10450_map_associations', '10327_map_associations', '10326_map_associations', '10200_map_associations', '10199_map_associations',
             '10198_map_associations', '10072_map_associations', '10071_map_associations']

In [ ]:
cat = GCRCatalogs.load_catalog("cosmoDC2_v1.1.4_small")
cosmodc2_data = cat.get_quantities(['ra', 'dec', 'mag_i', 'mag_r', 'mag_z', 'halo_id', 'halo_mass', 'position_x', 'position_y', 'redshift', 'galaxy_id', 'stellar_mass'], 
                                 filters=['halo_mass>1e13'])

In [ ]:
#Working with one tile at the moment (with prob cut)
amico_data = Table.read('/sps/lsst/groups/clusters/amico_validation_project/catalogs/amico_cats/amico_map_associations_uid/9943_map_associations.fits')
#amico_data2 = Table.read('/sps/lsst/groups/clusters/amico_validation_project/catalogs/amico_cats/amico_map_associations_cut/9943_map_associations.fits')
"""
for name in cat_names[1:]:
    cat_path = '/sps/lsst/groups/clusters/amico_validation_project/catalogs/amico_map_associations/' + name + '.fits'
    amico_data1 = Table.read(cat_path)
    amico_data = vstack([amico_data, amico_data1])
    """

"\nfor name in cat_names[1:]:\n    cat_path = '/sps/lsst/groups/clusters/amico_validation_project/catalogs/amico_map_associations/' + name + '.fits'\n    amico_data1 = Table.read(cat_path)\n    amico_data = vstack([amico_data, amico_data1])\n    "

In [55]:
"""cl_list1 = [] #prend une plombe
for cl in cosmodc2_data['halo_id']:
    if cl not in cl_list1:
        cl_list1.append(cl)"""
cl_list1 = np.loadtxt('/pbs/home/n/namourou/test_jupyter/cluster_challenge/notebooks/outfiles_1013/membership_matching/cl_list_cosmodc2.txt')
uid_list_am = []
for uid in amico_data['uid']:
    if uid not in uid_list_am:
        uid_list_am.append(uid)
#cl_list_am2 = []
#for cl2 in amico_data2['halo_id']:
#    if cl2 not in cl_list_am2:
#        cl_list_am2.append(cl2)


In [56]:
c1 = ClCatalog('Cat1', id=list(map(int, cl_list1)), mass =  1e14*np.arange(len(cl_list1))*10)
c2 = ClCatalog('Cat2', id=list(map(int, uid_list_am)), mass =  1e14*np.arange(len(uid_list_am))*10)

In [57]:
input1 = Table({'ID' : cosmodc2_data['halo_id'].astype(int), 'MASS' : cosmodc2_data['halo_mass']})

id,mass,mt_self,mt_other,mt_multi_self,mt_multi_other
str21,float64,object,object,object,object
3,59.768822,None,None,[],[]
14,30.979462,None,None,[],[]
18,25.938002,None,None,[],[]
19,22.818371,None,None,[],[]
22,20.396503,None,None,[],[]
27,23.655783,None,None,[],[]
28,59.101021,None,None,[],[]
29,35.639301,None,None,[],[]
34,21.288181,None,None,[],[]


In [58]:
input2 = Table({'ID' : amico_data['uid']})
input3 = Table({'ID' : amico_data2['halo_id']})
#input2['richness'] = 1e14*np.arange(1, 5)*10
#display(input1)
#display(input2)

In [59]:
input1_mem = Table({'ID' : cosmodc2_data['galaxy_id'].astype(int), 'ID_CLUSTER' : cosmodc2_data['halo_id'].astype(int)})

In [60]:
input2_mem = Table({'ID' : amico_data['mb_id'].astype(int), 'ID_CLUSTER' : amico_data['uid'].astype(int)})
input3_mem = Table({'ID' : amico_data2['mb_id'].astype(int), 'ID_CLUSTER' : amico_data2['halo_id'].astype(int)})

In [61]:
input1_mem['RA'] = cosmodc2_data['ra']
#input2_mem['RA'] = np.arange(len(input2_mem))*10.0
input1_mem['DEC'] = cosmodc2_data['dec']
#input2_mem['DEC'] = 0.0
input1_mem['Z'] = cosmodc2_data['redshift']
#input2_mem['Z'] = 0.1
input1_mem['PMEM'] = 1.0

In [62]:
input2_mem['PMEM'] = amico_data['prob']
input3_mem['PMEM'] = amico_data2['prob']

In [63]:
c1.add_members(id=input1_mem['ID'].astype(int), id_cluster=input1_mem['ID_CLUSTER'].astype(int),
               ra=input1_mem['RA'], dec=input1_mem['DEC']['PMEM')

/sps/lsst/users/namourou/conda_clone_210223/descv1.1/clevar/clevar/catalog.py:1032: UserWarning: Some galaxies were not members of the cluster catalog. They are stored in leftover_members attribute.
  warnings.warn(


In [64]:
c2.add_members(id=input2_mem['ID'], id_cluster=input2_mem['ID_CLUSTER'],
               pmem=input2_mem['PMEM'])
c3.add_members(id=input3_mem['ID'], id_cluster=input3_mem['ID_CLUSTER'],
               pmem=input3_mem['PMEM'])

In [65]:
from clevar.match import MembershipMatch
mt = MembershipMatch()

In [66]:
match_config = {
    'type': 'cross', # options are cross, cat1, cat2
    'preference': 'shared_member_fraction', # other options are more_massive, angular_proximity or redshift_proximity
    'minimum_share_fraction': 0,
    'match_members_kwargs': {'method':'id'},
}

In [36]:
mt.match_from_config(c1, c3, match_config)

In [67]:
mt.match_from_config(c1, c2, match_config)

10,550 members were matched.

## Multiple match (catalog 1)
Finding candidates (Cat1)
* 619/61,725 objects matched.es

## Multiple match (catalog 2)
Finding candidates (Cat2)
* 519/642 objects matched.

## Finding unique matches of catalog 1
Unique Matches (Cat1)
* 442/61,725 objects matched.

## Finding unique matches of catalog 2
Unique Matches (Cat2)
* 436/642 objects matched.
Cross Matches (Cat1)
* 435/61,725 objects matched.
Cross Matches (Cat2)
* 435/642 objects matched.


In [72]:
large_amico_data = Table.read('/sps/lsst/groups/clusters/amico_validation_project/catalogs/amico_cats/amico_map_associations_uid/' + '9943' + '_map_associations.fits')
for name in tile_names[1:]:
    amico_data2 = Table.read('/sps/lsst/groups/clusters/amico_validation_project/catalogs/amico_cats/amico_map_associations_uid/' + name + '_map_associations.fits')
    large_amico_data = vstack([large_amico_data, amico_data2])

In [73]:
cl_list_am = []
for cl in large_amico_data['uid']:
    if cl not in cl_list_am:
        cl_list_am.append(cl)

In [76]:
#c2 = ClCatalog('Cat2', id=input2['ID'])
c2 = ClCatalog('Cat2', id=cl_list_am, mass =  1e14*np.arange(len(cl_list_am))*10)

In [77]:
input2_mem = Table({'ID' : large_amico_data['mb_id'], 'ID_CLUSTER' : large_amico_data['uid'], 'PMEM' : large_amico_data['prob']})

In [78]:
c2.add_members(id=input2_mem['ID'], id_cluster=input2_mem['ID_CLUSTER'],
               pmem=input2_mem['PMEM'])

In [79]:
from clevar.match import MembershipMatch
mt = MembershipMatch()

In [80]:
match_config = {
    'type': 'cross', # options are cross, cat1, cat2
    'preference': 'shared_member_fraction', # other options are more_massive, angular_proximity or redshift_proximity
    'minimum_share_fraction': 0,
    'match_members_kwargs': {'method':'id'},
}

In [83]:
mt.match_from_config(c1, c2, match_config)

380,516 members were matched.

## Multiple match (catalog 1)
Finding candidates (Cat1)
* 12,314/61,726 objects matched.s

## Multiple match (catalog 2)
Finding candidates (Cat2)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




## Finding unique matches of catalog 1
Unique Matches (Cat1)
* 10,819/61,726 objects matched.

## Finding unique matches of catalog 2
Unique Matches (Cat2)
* 9,622/32,735 objects matched.
Cross Matches (Cat1)
* 8,834/61,726 objects matched.
Cross Matches (Cat2)
* 8,413/32,735 objects matched.


In [84]:
c1.write('/pbs/home/n/namourou/test_jupyter/cluster_challenge/notebooks/outfiles/membermatching/' + 'c1_new.fits', overwrite=True)
c2.write('/pbs/home/n/namourou/test_jupyter/cluster_challenge/notebooks/outfiles/membermatching/' + 'c2_new.fits', overwrite = True)

In [85]:
mt.save_matches(c1, c2, out_dir='/pbs/home/n/namourou/test_jupyter/cluster_challenge/notebooks/outfiles/membermatching/', overwrite=True)

In [86]:
outpath = '/pbs/home/n/namourou/test_jupyter/cluster_challenge/notebooks/outfiles/membermatching/'

In [87]:
output_matched_catalog(outpath + 'c1_new.fits', outpath + 'c2_new.fits',outpath+'output_catalog.fits', c1, c2, matching_type='cross', overwrite=True) # Saves matchings in one file

In [90]:
cdc = ClCatalog.read_full(outpath + 'c1_new.fits') #cosmoDC2_small matched table
cam = ClCatalog.read_full(outpath + 'c2_new.fits') #amico matched table
c_merged = ClCatalog.read(outpath +'output_catalog.fits', 'merged', full = True)

    * ClEvar used in matching: 0.13.8
    * ClEvar used in matching: 0.13.8
